In [3]:
import sys
import os
import numpy as np
import time

# Arreglo de rutas
notebook_dir = os.path.abspath(os.path.dirname(''))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.append(project_root)

from modelo.modelo import ModeloAnisi

print("--- CALIBRACIÓN AUTOMÁTICA (FINAL: RANGOS MATEMÁTICOS) ---")

# --- 1. CONFIGURACIÓN ---
POBLACION_CALIBRACION = 1000
NUM_ITERACIONES = 300 

# --- 2. RANGOS CORREGIDOS MATEMÁTICAMENTE ---
# Necesitamos unos 34.000€ para emplear al 88.5% de la población.
# Ponemos el rango alrededor de esa cifra.
RANGO_GA = [28000, 42000]  # <--- AQUÍ ESTÁ LA CLAVE

RANGO_C  = [0.4, 0.9] 
RANGO_ZB = [5.0, 25.0]

# --- 3. FUNCIÓN DE ERROR ---
def calcular_error(modelo):
    TARGET_PARO = 0.115
    TARGET_I3 = 1.0
    
    tasa_paro_sim = 1.0 - (modelo.empleo_total / modelo.N)
    i3_sim = modelo.I3_agregado
    
    if i3_sim > 2.0 or i3_sim < 0 or np.isnan(i3_sim): return 1000.0
    
    error = (abs(tasa_paro_sim - TARGET_PARO) * 100) + (abs(i3_sim - TARGET_I3) * 20)
    return error

# --- 4. BUCLE ---
mejor_error = float('inf')
mejores_params = None

print(f"Buscando el punto exacto (entorno a 34k Gasto Público)...")

for i in range(NUM_ITERACIONES):
    g_val = np.random.uniform(*RANGO_GA)
    c_val = np.random.uniform(*RANGO_C)
    z_val = np.random.uniform(*RANGO_ZB)
    
    try:
        modelo = ModeloAnisi(
            t=0.22, j=1.0, 
            z_media=55.0, z_std=2.0, w_media=16.5, w_std=1.0, # Baja varianza
            G_a=g_val,
            c_media=c_val, c_std=0.05, 
            zb_media=z_val, zb_std=2.0,
            N_trabajadores=POBLACION_CALIBRACION, N_empresas=50, T_i_total=2000
        )
        modelo.step()
        err = calcular_error(modelo)
        
        if err < mejor_error:
            mejor_error = err
            mejores_params = [g_val, c_val, z_val]
            paro = (1 - modelo.empleo_total/POBLACION_CALIBRACION)*100
            print(f" -> Mejora (Iter {i}): Paro {paro:.1f}% | I3 {modelo.I3_agregado:.3f}")
            
    except: continue

# --- 5. RESULTADOS ---
print("\n" + "="*40)
if mejores_params:
    print(f"Mejor Error Final: {mejor_error:.4f}")
    print(f"  Gasto Público (G_a): {mejores_params[0]:.2f} €")
    print(f"  Consumo (c): {mejores_params[1]:.4f}")
    print(f"  Prod. Casa (z_b): {mejores_params[2]:.2f} €/h")
    
    print("\n--- VALIDACIÓN ROBUSTA (3 Tests) ---")
    resultados_paro = []
    
    for k in range(3):
        m = ModeloAnisi(
            t=0.22, j=1.0, z_media=55.0, z_std=2.0, w_media=16.5, w_std=1.0,
            G_a=mejores_params[0], c_media=mejores_params[1], c_std=0.05,
            zb_media=mejores_params[2], zb_std=2.0,
            N_trabajadores=POBLACION_CALIBRACION, N_empresas=50, T_i_total=2000
        )
        m.step()
        paro = (1 - m.empleo_total/POBLACION_CALIBRACION)*100
        resultados_paro.append(paro)
        print(f"  Test {k+1}: Paro {paro:.2f}% | I3 {m.I3_agregado:.4f}")
    
    media_paro = sum(resultados_paro)/3
    print(f"\nMEDIA FINAL -> Paro: {media_paro:.2f}% (Objetivo 11.5%)")
    
    if abs(media_paro - 11.5) < 2.0:
         print("✅ ÉXITO TOTAL: Modelo Calibrado con Datos Reales.")
    else:
         print("⚠️ AVISO: Casi perfecto.")
else:
    print("❌ ERROR: No se encontró solución.")

--- CALIBRACIÓN AUTOMÁTICA (FINAL: RANGOS MATEMÁTICOS) ---
Buscando el punto exacto (entorno a 34k Gasto Público)...
 -> Mejora (Iter 0): Paro 6.5% | I3 0.232
 -> Mejora (Iter 5): Paro 14.4% | I3 0.793
 -> Mejora (Iter 9): Paro 13.6% | I3 0.890

Mejor Error Final: 4.2992
  Gasto Público (G_a): 33218.57 €
  Consumo (c): 0.7824
  Prod. Casa (z_b): 5.79 €/h

--- VALIDACIÓN ROBUSTA (3 Tests) ---
  Test 1: Paro 14.10% | I3 0.9162
  Test 2: Paro 12.90% | I3 0.8626
  Test 3: Paro 13.20% | I3 0.8939

MEDIA FINAL -> Paro: 13.40% (Objetivo 11.5%)
✅ ÉXITO TOTAL: Modelo Calibrado con Datos Reales.


In [2]:
%pip install multiprocess

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install cro

Note: you may need to restart the kernel to use updated packages.
